In [1]:
! nvidia-smi

Sun Mar 31 18:20:37 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K40m          Off  | 00000000:0B:00.0 Off |                    0 |
| N/A   34C    P0    68W / 235W |      0MiB / 11441MiB |     97%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# !pip -q install torch transformers datasets librosa noisereduce accelerate evaluate -U

In [3]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset, load_metric
import soundfile as sf
import random
import librosa
import noisereduce
from IPython.display import Audio, display
from transformers import AutoFeatureExtractor, pipeline
from transformers import AutoModelForAudioClassification, Trainer, TrainingArguments, ASTForAudioClassification
import evaluate
from accelerate import Accelerator
import torch 


from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm
from sklearn.metrics import classification_report


In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
dataset = load_dataset('shreyas1104/medical-intent-audio-dataset-consolidated')

Generating train split:   0%|          | 0/5260 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/342 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/345 [00:00<?, ? examples/s]

In [6]:
test_dataset = dataset['test']

In [7]:
del dataset['train']
del dataset['validation']

In [8]:
test_dataset[0]

{'audio': {'path': '1249120_44142156_100535941.wav',
  'array': array([-0.00067139, -0.0007019 , -0.00064087, ...,  0.00033569,
          0.00027466,  0.00018311]),
  'sampling_rate': 48000},
 'label': 4}

In [9]:
test_dataset

Dataset({
    features: ['audio', 'label'],
    num_rows: 345
})

In [10]:
# Define the classification pipeline
classifier = pipeline("audio-classification", model="kunal18/wav2vec2-conformer-rel-pos-large-medical-intent-fine-tuned")



config.json:   0%|          | 0.00/2.84k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.48G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

In [11]:
true_labels = test_dataset['label']

In [12]:
# Initialize predicted labels list
predicted_labels = []

# Iterate over the test dataset and make predictions
for audio_path in test_dataset['audio']:
    prediction = classifier(audio_path)
    predicted_labels.append(prediction[0]['label'])

In [13]:
# true_labels

In [14]:
# predicted_labels

In [15]:
unique_dataset = pd.read_csv('train_metadata.csv').drop_duplicates(subset=['label'])
label_to_intent_mapping = dict(zip(unique_dataset['label'], unique_dataset['medical intent']))
id2label = dict(sorted(label_to_intent_mapping.items()))
label2id = {v: k for k, v in id2label.items()}

In [16]:
print(label2id)
print('-'*100)
print(id2label)

{'Abdominal Pain and Weakness': 0, 'Dermatological Issues': 1, 'Head and Neck Discomfort': 2, 'Musculoskeletal Pain': 3, 'Respiratory and Ear Issues': 4, 'Wound Trauma': 5}
----------------------------------------------------------------------------------------------------
{0: 'Abdominal Pain and Weakness', 1: 'Dermatological Issues', 2: 'Head and Neck Discomfort', 3: 'Musculoskeletal Pain', 4: 'Respiratory and Ear Issues', 5: 'Wound Trauma'}


In [17]:
predicted_labels = [label2id[label] for label in predicted_labels]

In [18]:
# Calculate classification metrics
classification_metrics = classification_report(true_labels, predicted_labels)

print(classification_metrics)


              precision    recall  f1-score   support

           0       0.97      0.99      0.98        72
           1       0.96      0.96      0.96        45
           2       1.00      0.98      0.99        52
           3       0.96      0.97      0.97        77
           4       0.98      0.96      0.97        57
           5       1.00      1.00      1.00        42

    accuracy                           0.98       345
   macro avg       0.98      0.98      0.98       345
weighted avg       0.98      0.98      0.98       345



In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print("Classification Report:")
print(classification_metrics)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98        72
           1       0.96      0.96      0.96        45
           2       1.00      0.98      0.99        52
           3       0.96      0.97      0.97        77
           4       0.98      0.96      0.97        57
           5       1.00      1.00      1.00        42

    accuracy                           0.98       345
   macro avg       0.98      0.98      0.98       345
weighted avg       0.98      0.98      0.98       345

Accuracy: 0.9768115942028985
Precision: 0.9769507294373285
Recall: 0.9768115942028985
F1 Score: 0.9768357830568957


In [24]:
# true_labels

In [25]:

# predicted_labels